**Cài đặt thư viện transformers để sử dụng BERT cho bài toán hỏi/đáp. Thông tin về thư viện transformers (HuggingFace) có thể được tham khảo tại đây: https://huggingface.co/docs/transformers/installation**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.3 MB/s 
     |████████████████████████████████| 182 kB 58.3 MB/s 
     |████████████████████████████████| 7.6 MB 58.2 MB/s 


**Import các thư viện cần thiết**

In [ ]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

**Tiến hành khởi tạo và khai báo các tham số cho mô hình hỏi đáp với BERT, trong đó:**

*   **Hỏi/đáp: sử dụng mô hình BERT đã được huấn luyện (pre-trained model) trên tập dữ liệu SQuAD (https://rajpurkar.github.io/SQuAD-explorer/)**
*   **Tách từ: tương tự như mô hình hỏi/đáp - chúng ta cũng khai báo cho BERT sử dụng mô hình đã được huấn luyện (pre-trained model) trên tập dữ liệu SQuAD**



In [ ]:
# Mô hình hỏi/đáp BERT đã được pre-trained trên tập dữ liệu SQuAD
bertQA = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Mô hình tách từ (tokenizer) BERT đã được pre-trained trên tập dữ liệu SQuAD
bertTokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

**Tiến hành thử nghiệm cho mô hình BERT học và trả lời câu hỏi**

In [ ]:
# Danh mục các câu hỏi
question1 = '''Who is Nguyễn Du?'''
question2 = '''Where Nguyễn Du was born?'''
question3 = '''Who write The Tale of Kiều?'''

# Đoạn văn bản được dùng để huấn luyện mô hình BERT cho bài toán hỏi đáp với 3 câu hỏi bên trên
paragraph = '''Nguyễn Du (阮攸; 3 January 1766 – 16 September 1820), pen names Tố Như (素如) and Thanh Hiên (清軒), is a celebrated Vietnamese poet. 
          He is most known for writing the epic poem The Tale of Kiều. 
          Nguyễn Du was born in a great wealthy family in 1765 in Bích Câu, Thăng Long.[3][4][5] 
          His father's name is Nguyễn Nghiễm, who was born in Tiên Điền village, Nghi Xuân, Hà Tĩnh, Vietnam. 
          He was the seventh child of Nguyễn Nghiễm, a former prime minister under the Lê dynasty. 
          By the age of 10, Nguyễn lost his father, he also lost his mother at age 13, so for most of his teen years he lived with his brother Nguyễn Khản or with his brother-in-law Đoàn Nguyễn Tuấn.  
          At the age of 19 (some sources say 17), Nguyễn passed the provincial examination and received the title of "tú tài" (Bachelor's degree), which made him (very roughly) the equivalent of a high school graduate. 
          However, in Nguyễn Du's time this was a far more difficult credential to obtain both because few people were affluent enough to devote themselves to study and because of exacting standards applied.  
          Nguyễn's mother was his father's third wife, noted for her ability at singing and composing poetry. In fact, she made her living by singing, which at that time was considered a disreputable occupation. 
          It is said that Nguyễn may have inherited a part of his talents from his mother. 
          He loved listening to traditional songs; and there was a rumor that, when he was 18, he himself eloped with a songstress. '''

# Tiến hành học mô hình biểu diễn của các câu hỏi đi kèm với đoạn văn [paragraph]         
question1_encoding = bertTokenizer.encode_plus(text=question1,text_pair=paragraph)
question2_encoding = bertTokenizer.encode_plus(text=question2,text_pair=paragraph)
question3_encoding = bertTokenizer.encode_plus(text=question3,text_pair=paragraph)

# Lấy ra danh sách các token ids của câu hỏi và đoạn văn bản
# Ví dụ: [101, 2040, 2003, 16577, 4241, 1029, 102, 16577, 4241, 1006, 100, 100, 1025, 1017, 2254,...]
# Lưu ý, ids: 101 và 102 là các token đặc biệt [CLS], [SEP]
inputs = question1_encoding['input_ids']

# Tiến hành lấy ra phân đoạn giữa câu hỏi và đoạn văn bản
# Với các token trong câu hỏi sẽ có giá trị là 0 và đoạn văn sẽ là 1
# Ví dụ: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]
sentence_embedding = question1_encoding['token_type_ids']

# Lấy ra danh sách các từ/token gốc từ ids
# Ví dụ:
# ids: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]
# tokens: ['[CLS]', 'who', 'is', 'nguyen', 'du', '?', '[SEP]', 'nguyen', 'du', ...]
tokens = bertTokenizer.convert_ids_to_tokens(inputs)

**Tiến hành sử dụng mô hình BERT để tìm câu trả lời cho câu hỏi, dựa trên đoạn văn được cung cấp, thông quan [inputs] và [sentence_embedding]**

In [ ]:
output = bertQA(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

**Tiến hành rút trích ra câu trả lời trong đoạn văn [paragraph] dựa trên từ khóa bắt đầu và kết thúc**

In [ ]:
# Lấy ra từ khóa bắt đầu cho câu trả lời có trọng số cao nhất trong [start_logits]
answer_start = torch.argmax(output.start_logits)

# Lấy ra từ khóa kết thúc cho câu trả lời có trọng số cao nhất trong [end_logits]
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("Không tìn thấy câu trả lời")
    
print("Câu hỏi: [{}]".format(question1.capitalize()))
print("Câu trả lời: [{}].".format(answer.capitalize()))

Câu hỏi: [Who is nguyễn du?]
Câu trả lời: [A celebrated vietnamese poet].
